In [7]:
import pandas as pd
import numpy as np
import re

In [8]:
with open('SOC2020desc.csv', 'r') as file:
    df = pd.read_csv(file)
df.head()

,SOC\n2020 Major Group,SOC\n2020 Sub-Major Group,SOC\n2020 Minor Group,SOC 2020 Unit Group,SOC\n2020 \nGroup Title,Groups Classified Within Sub-Groups,Group Description,Typical Entry Routes And Associated Qualifications,Tasks,Related Job Titles
0,1,<blank>,<blank>,<blank>,"Managers, directors and senior officials",Occupations in this major group are classified...,This major group covers occupations whose task...,<blank>,<blank>,<blank>
1,<blank>,11,<blank>,<blank>,Corporate managers and directors,Occupations in this sub-major group are classi...,Job holders in this sub-major group formulate ...,<blank>,<blank>,<blank>
2,<blank>,<blank>,111,<blank>,Chief executives and senior officials,Occupations in this minor group are classified...,"Jobholders in this minor group plan, organise ...",<blank>,<blank>,<blank>
3,<blank>,<blank>,<blank>,1111,Chief executives and senior officials,<blank>,Chief executives and senior officials head lar...,Entry may be by appointment or internal promot...,"~analyses economic, social, legal and other da...",~Chairman\n\n~Chief executive\n\n~Civil servan...
4,<blank>,<blank>,<blank>,1112,Elected officers and representatives,<blank>,Elected representatives in national government...,Entry is by election.,~represents constituency within the legislatur...,~Councillor (local government)\n\n~Member of P...


In [9]:
# Extract 
job_titles = df[['SOC 2020 Unit Group', 'Related Job Titles']]
job_titles = job_titles.rename(columns={'SOC 2020 Unit Group': 'SOC_code', 'Related Job Titles': 'init_info'})
job_titles = job_titles[job_titles['init_info'] != '<blank>']

In [10]:
# After exploding, remove rows where 'Related Job Titles' is empty# Clean and split the 'Related Job Titles' column, then explode it into multiple rows
job_titles['init_info'] = (
    job_titles['init_info']
    # .str.replace(r'\s*\(.*?\)', '', regex=True)  # Remove text within parentheses
    .str.replace(r'[^\w\s]', '', regex=True)  # Remove all punctuation
    .str.strip()  # Remove leading and trailing spaces
    .str.split('\n\n')  # Split by double newlines
)

# Explode the column and remove empty rows
job_titles = job_titles.explode('init_info')
job_titles = job_titles[job_titles['init_info'].str.strip() != ''] 
job_titles = job_titles.reset_index(drop=True)

In [11]:
# Wrirte CSV file for job_titles
job_titles.to_csv('job_titles.csv', index=False)

In [12]:
# Create JSON file for job_titles['init_info']
job_titles_json = job_titles['init_info'].to_json(orient='records')
with open('job_titles.json', 'w') as file:
    file.write(job_titles_json)